In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
import sys
from datetime import date, timedelta

In [2]:
spark = SparkSession \
    .builder \
    .appName("Brewery") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/30 23:06:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.parquet("silver/")

In [4]:
df.show()

+--------------------+--------------+--------------+------------+----------------+-------------+--------------------+----------------+-----------------+--------------------+-------------+-------------------+-------------------+--------------------+--------------------+----------+-------------+
|      name_address_1|name_address_2|name_address_3|brewery_type|       name_city| name_country|          id_brewery|latitude_brewery|longitude_brewery|        name_brewery|phone_brewery|postal_code_brewery|name_state_province|         name_street|         website_url|   dt_load|   name_state|
+--------------------+--------------+--------------+------------+----------------+-------------+--------------------+----------------+-----------------+--------------------+-------------+-------------------+-------------------+--------------------+--------------------+----------+-------------+
|       62970 18th St|          NULL|          NULL|       large|            Bend|United States|6d14b220-8926-452..

In [5]:
df = df.withColumn('index',row_number().over(Window.partitionBy(col("id_brewery")).orderBy(col("dt_load").desc())))
df = df.filter(df.index == 1) 
    
df = df.drop("index")

In [6]:
df.groupBy(df.brewery_type, df.name_state).count().show()

+------------+--------------+-----+
|brewery_type|    name_state|count|
+------------+--------------+-----+
|       micro|     Wisconsin|    2|
|      closed|    California|    1|
|       large|        Oregon|    4|
|      closed|    Washington|    1|
|       micro|      Virginia|    1|
|       micro| Massachusetts|    1|
|       micro|          Iowa|    1|
|       micro|      Oklahoma|    1|
|       micro|      Michigan|    1|
|       micro|North Carolina|    1|
|     brewpub|      New York|    2|
|       micro|     Minnesota|    3|
|       micro|      Colorado|    1|
|       micro|         Laois|    1|
|       micro|  Pennsylvania|    1|
|       micro|    California|    2|
|       micro|        Nevada|    1|
|       micro|       Vermont|    1|
|     brewpub|South Carolina|    1|
|       large|      Colorado|    1|
+------------+--------------+-----+
only showing top 20 rows



In [7]:
df = df.groupBy(df.brewery_type, df.name_state).count()

In [8]:
df = df.withColumnRenamed("count", "qty_brewery_type_and_location")

In [9]:
today = date.today()

In [11]:
df.write.format("parquet").mode("append")\
        .option("mergeSchema", "true").save(f'gold/day{today}')